
# TP : Prédire le prix de location de logements à Antananarivo avec une régression linéaire multiple

## 🎯 Objectifs pédagogiques

- Appliquer un pipeline de prétraitement complet sur un jeu de données semi-structuré.
- Gérer les variables qualitatives, les valeurs manquantes, la multicolinéarité et la scalabilité.
- Construire, tester et évaluer un modèle de régression linéaire multiple.
- Déployer le modèle dans une application Python Streamlit avec interface utilisateur.

## 🗂️ Jeu de données

Le jeu de données doit être collecté ou scrappé dans les pages comme Facebook. Il doit comporter les colonnes suivantes :

- `quartier` (catégorielle)
- `superficie` (numérique)
- `nombre_chambres` (numérique)
- `douche_wc`(interieur ou exterieur)
- `type_d_acces` (sans, moto, voiture, voiture_avec_par_parking)
- `meublé` (booléen:  oui ou non)
- `état_général` (catégorielle : bon, moyen, mauvais)
- `loyer_mensuel` (target)

## 🧪 Étapes du TP

### 📌 Partie 1 : Préparation des données
- Lecture du dataset brut
- Gestion des valeurs manquantes
- Encodage des variables catégorielles
- Création de variables dérivées
- Détection et suppression des variables fortement corrélées
- Standardisation et normalisation

### 📌 Partie 2 : Modélisation

- Séparation train/test
- Implémentation de la régression linéaire multiple
- Évaluation : R², RMSE
- Vérification des hypothèses d'élligibilité de la régression linéaire multiple (surtout sur les erreurs)

### 📌 Partie 3 : Optimisation du modèle

- Sélection de variables : backward elimination, RFE (à documenter)

### 📌 Partie 4 : Déploiement d’une application Streamlit

- Interface de saisie utilisateur
- Affichage du loyer prédit
- Visualisation des poids des variables
- Affichage sur la carte interactive

## 🧭 Carte interactive (option avancée)

Utiliser `streamlit-folium` pour permettre à l’utilisateur de cliquer sur une carte et de récupérer les coordonnées GPS. À partir de ces coordonnées, déterminer automatiquement le quartier en utilisant un fichier GeoJSON ou un système de polygones avec `geopandas`.

## 🔧 Technologies à utiliser

- `pandas`, `numpy`, `scikit-learn`, `matplotlib`, `seaborn`, `joblib`
- `streamlit`, `folium`, `streamlit-folium`
- Optionnel : `geopandas`, `shapely`

## 💡 Bonus

- Carte interactive avec folium
- Simuler des données additionnelles (pollution, sécurité)
- Tri automatique des caractéristiques influentes




## 📌 Partie 1 : Préparation des données

In [ ]:
import math

# Lecture du dataset brut
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

maison = pd.read_csv("Location de maison Antananarivo  - Données finales - 1.csv")

maison_2 = pd.read_csv("Location de maison Antananarivo  - Données finales - 1.csv")


In [ ]:
maison

In [ ]:
maison.dtypes

In [ ]:
# Transform superficie into float
maison['superficie'] = maison['superficie'].apply(lambda x: x.replace(",", ".") if type(x) == str else x).astype(float)

In [ ]:
maison.dtypes

In [ ]:
maison['meublé'] = maison['meublé'].apply(lambda x: "oui" if x == "True" else ("non" if x == "False" else x))

In [ ]:
maison

In [ ]:
range_etat_general = [
    ("mauvais", 0, 300000),
    ("moyen", 300000, 800000),
    ("bon", 800000, math.inf)
]
maison['état_général'] = maison['loyer_mensuel'].apply(lambda x: range_etat_general[0][0] if range_etat_general[0][1] <= x < range_etat_general[0][2] else (range_etat_general[1][0] if range_etat_general[1][1] <= x < range_etat_general[1][2] else range_etat_general[2][0]))

In [ ]:
maison

In [ ]:
# Encoding One-hot douche_wc
one_hot = pd.get_dummies(maison['douche_wc'])
maison = one_hot.join(maison)
maison = maison.drop('douche_wc',axis = 1)

In [ ]:
maison

In [ ]:
quantile = maison['superficie'].quantile([0.25,0.5,0.75])

In [ ]:
quantile

In [ ]:
# Remplir les valeurs manquantes superficie
maison['superficie'] = maison['superficie'].fillna(maison.apply(lambda x: quantile[0.25] if x['état_général'] == "mauvais" else (quantile[0.5] if x['état_général'] == "moyen" else quantile[0.75]), axis = 1))

In [ ]:
# Remplir les valeurs manquantes loyer_mensuel
maison['loyer_mensuel'] = maison['loyer_mensuel'].fillna(maison['loyer_mensuel'].mean())

In [ ]:
Q1 = quantile[0.25]
Q3 = quantile[0.75]
IQR_superficie = Q3 - Q1
# Define the lower and upper thresholds
lower_bound = Q1 - 1.5 * IQR_superficie
upper_bound = Q3 + 1.5 * IQR_superficie

maison = maison[(maison['superficie'] > lower_bound) & (maison['superficie'] < upper_bound)]

In [ ]:
maison

In [ ]:
quantile_loyer_mensuel = maison['loyer_mensuel'].quantile([0.25,0.5,0.75])

In [ ]:
quantile_loyer_mensuel

In [ ]:
Q1_loyer = quantile_loyer_mensuel[0.25]
Q3_loyer = quantile_loyer_mensuel[0.75]
IQR_loyer = Q3_loyer - Q1_loyer
# Define the lower and upper thresholds
lower_bound = Q1_loyer - 1.5 * IQR_loyer
upper_bound = Q3_loyer + 1.5 * IQR_loyer

maison = maison[(maison['loyer_mensuel'] > lower_bound) & (maison['loyer_mensuel'] < upper_bound)]

In [ ]:
maison

In [ ]:
# Encoding Ondinal état_général
mapping = {
    "bon": 3,
    "moyen": 2,
    "mauvais": 1
}
pd.set_option('future.no_silent_downcasting', True)
maison['état_général'] = maison['état_général'].replace(mapping)

In [ ]:
maison

In [ ]:
# Encoding One-hot type_d_acces
one_hot = pd.get_dummies(maison['type_d_acces'])
maison = one_hot.join(maison)
maison = maison.drop('type_d_acces',axis = 1)

In [ ]:
maison

In [ ]:
# Encoding One-hot type_d_acces
maison['meublé'] = maison['meublé'].fillna("non")
mapping = {
    "oui": 2,
    "non": 1
}
maison['meublé'] = maison['meublé'].replace(mapping)

In [ ]:
maison

In [ ]:
maison_without_quartier = maison.loc[:, maison.columns != 'quartier']

In [ ]:
maison_without_quartier

In [ ]:
correlation = maison_without_quartier.corr()

In [ ]:
correlation = correlation['loyer_mensuel'].abs().sort_values()

In [ ]:
correlation

In [ ]:
def superficie_into_float(df):
    df['superficie'] = df['superficie'].apply(lambda x: x.replace(",", ".") if type(x) == str else x).astype(float)
    return df

def meuble_into_oui_non(df):
    df['meublé'] = df['meublé'].apply(lambda x: "oui" if x == "True" else ("non" if x == "False" else x))
    return df

def loyer_mensuel_fillna(df):
    df['loyer_mensuel'] = df['loyer_mensuel'].fillna(df['loyer_mensuel'].mean())
    return df

def etat_general_into_bon_mauvais_moyen(df):
    etat_general = [
        ("mauvais", 0, 300000),
        ("moyen", 300000, 800000),
        ("bon", 800000, math.inf)
    ]
    df['état_général'] = df['loyer_mensuel'].apply(lambda x: etat_general[0][0] if etat_general[0][1] <= x < etat_general[0][2] else (etat_general[1][0] if etat_general[1][1] <= x < etat_general[1][2] else etat_general[2][0]))
    return df

def douche_wc_separate(df):
    one_hot_douche_wc = pd.get_dummies(df['douche_wc'])
    df = one_hot_douche_wc.join(df)
    return df.drop('douche_wc',axis = 1)

def superficie_fillna(df):
    quantile_superficie = df['superficie'].quantile([0.25,0.5,0.75])
    df['superficie'] = df['superficie'].fillna(df.apply(lambda x: quantile_superficie[0.25] if x['état_général'] == "mauvais" else (quantile_superficie[0.5] if x['état_général'] == "moyen" else quantile_superficie[0.75]), axis = 1))
    return df

def aberrante_value_superficie(df):
    quantile_superficie = df['superficie'].quantile([0.25,0.5,0.75])
    Q1_superficie = quantile_superficie[0.25]
    Q3_superficie = quantile_superficie[0.75]
    IQR_super = Q3_superficie - Q1_superficie
    # Define the lower and upper thresholds
    lower_bound_superficie = Q1 - 1.5 * IQR_super
    upper_bound_superficie = Q3 + 1.5 * IQR_super
    return df[(df['superficie'] > lower_bound_superficie) & (df['superficie'] < upper_bound_superficie)]

def aberrante_value_loyer_mensuel(df):
    quantile_loyer = df['loyer_mensuel'].quantile([0.25,0.5,0.75])
    Q1_loyer_mensuel = quantile_loyer[0.25]
    Q3_loyer_mensuel = quantile_loyer[0.75]
    IQR_loyer_mensuel = Q3_loyer_mensuel - Q1_loyer_mensuel
    # Define the lower and upper thresholds
    lower_bound_loyer_mensuel = Q1_loyer_mensuel - 1.5 * IQR_loyer_mensuel
    upper_bound_loyer_mensuel = Q3_loyer_mensuel + 1.5 * IQR_loyer_mensuel
    return df[(df['loyer_mensuel'] > lower_bound_loyer_mensuel) & (df['loyer_mensuel'] < upper_bound_loyer_mensuel)]

def etat_general_into_numerical(df):
    mapping_etat_general = {
        "bon": 3,
        "moyen": 2,
        "mauvais": 1
    }
    pd.set_option('future.no_silent_downcasting', True)
    df['état_général'] = df['état_général'].replace(mapping_etat_general)
    return df


def type_d_acces_separate(df):
    one_hot_type_acces = pd.get_dummies(df['type_d_acces'])
    df = one_hot_type_acces.join(df)
    return df.drop('type_d_acces',axis = 1)

def meuble_into_numerical(df):
    df['meublé'] = df['meublé'].fillna("non")
    mapping_meuble = {
        "oui": 2,
        "non": 1
    }
    df['meublé'] = df['meublé'].replace(mapping_meuble)
    return df

def quartier_remove(df):
    return df.loc[:, df.columns != 'quartier']

def normalisation(df):
    correlation_norm = df.corr()
    correlation_norm = correlation_norm[target].abs().sort_values()
    strong_corr_norm = correlation_norm[(correlation_norm > 0.3)]
    corr_math_norm = df[strong_corr_norm.index].corr()
    features_normalisation = corr_math_norm.index
    print(features_normalisation)
    return (df[features_normalisation].astype(float) - df[features_normalisation].min().astype(float)) / (df[features_normalisation].max().astype(float) - df[features_normalisation].min().astype(float))

def normalisation_with_great_var(df):
    last_variance_sorted = df.var().sort_values()
    last_columns = last_variance_sorted[(last_variance_sorted > 0.05)].index
    print(last_columns)
    return df[last_columns]

def get_features(df):
    return df.iloc[:, df.columns != target]

def pre_treatment(df):
    df_pre_trait = superficie_into_float(df)
    df_pre_trait = meuble_into_oui_non(df_pre_trait)
    df_pre_trait = loyer_mensuel_fillna(df_pre_trait)
    df_pre_trait = etat_general_into_bon_mauvais_moyen(df_pre_trait)
    df_pre_trait = douche_wc_separate(df_pre_trait)
    df_pre_trait = superficie_fillna(df_pre_trait)
    df_pre_trait = aberrante_value_superficie(df_pre_trait)
    df_pre_trait = aberrante_value_loyer_mensuel(df_pre_trait)
    df_pre_trait = etat_general_into_numerical(df_pre_trait)
    df_pre_trait = type_d_acces_separate(df_pre_trait)
    df_pre_trait = meuble_into_numerical(df_pre_trait)
    y_targ = df_pre_trait[target]
    df_pre_trait = quartier_remove(df_pre_trait)
    df_pre_trait = normalisation(df_pre_trait)
    df_pre_trait = get_features(df_pre_trait)
    return normalisation_with_great_var(df_pre_trait), y_targ

In [ ]:
import seaborn as sns
strong_corr = correlation[(correlation > 0.3)]
corrmat = maison_without_quartier[strong_corr.index].corr()
sns.heatmap(corrmat)

In [ ]:
# Normalisation
target = "loyer_mensuel"
features = corrmat.drop([target]).index
maison_normalisation = (maison_without_quartier[features].astype(float) - maison_without_quartier[features].min().astype(float)) / (maison_without_quartier[features].max().astype(float) - maison_without_quartier[features].min().astype(float))

In [ ]:
maison_normalisation.var().sort_values()

In [ ]:
variance_sorted = maison_normalisation.var().sort_values()

In [ ]:
variance_sorted

In [ ]:
columns = variance_sorted[(variance_sorted > 0.05)].index
maison_normalisation = maison_normalisation[columns]

In [ ]:
columns

In [ ]:
maison_normalisation

In [ ]:
sns.heatmap(maison_normalisation.corr())

In [ ]:
columns

In [ ]:
scaler = StandardScaler()
maison_features = maison_without_quartier.loc[:, maison_without_quartier.columns != target]
maison_standardization = scaler.fit_transform(maison_features)

In [ ]:
maison_standardization

In [ ]:
# Valeur expliquée
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

y = maison[target]
X_standard = maison_standardization

In [ ]:
y

In [ ]:
lr = LinearRegression()
predict_standard = lr.fit(X_standard, y)
y_pred = lr.predict(X_standard)

In [ ]:
lr.predict(X_standard)

In [ ]:
predict_standard.score(X_standard, y)

In [ ]:
np.sqrt(mean_squared_error(y, y_pred))

## 📌 Partie 2 : Modélisation

In [ ]:
class MyLinearRegression:
    def __init__(self):
        self.theta_n = None
        self.theta_0 = None

    def fit(self, x_var, y_real):
        # Add bias term (column of 1s) to X
        X_b = np.c_[np.ones((x_var.shape[0], 1)), x_var]  # Add x0 = 1 to each instance
        # Normal Equation: theta = (X^T X)^(-1) X^T y

        theta = np.linalg.inv(X_b.T.dot(X_b)).dot(X_b.T).dot(y_real)
        self.theta_0 = theta[0]
        self.theta_n = theta[1:]

    def predict(self, x_var):
        # Add bias term (column of 1s) to X
        X_b = np.c_[np.ones((x_var.shape[0], 1)), x_var] # Add x0 = 1 to each instance
        # np.r_ (Merge the first Vector [0] with the Vector [1])
        return X_b.dot(np.r_[self.theta_0, self.theta_n])

    def get_r2_score(self, x_var, y_real):
        """
        Formule: 1 - [sum((y[i] - y_predicted[i]) ^ 2) - sum((y[i] - y_mean) ^ 2)]
        """
        y_predicted = self.predict(x_var)
        ss_reg = np.sum((y_real - y_predicted) ** 2)
        ss_tot = np.sum((y_real - np.mean(y_real)) ** 2)
        return 1 - (ss_reg / ss_tot)

    def get_rmse(self, x_var, y_real):
        """
        Formule: np.sqrt((1 / n) * sum((y[i] - y_predicted[i]) ^ 2))
        """
        y_predicted = self.predict(x_var)
        return np.sqrt(np.sum((y_real - y_predicted) ** 2) / y_real.shape[0])

In [ ]:
X = maison_normalisation.iloc[:, maison_normalisation.columns != target]

In [ ]:
# Split train, test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)

In [ ]:
lr_2 = MyLinearRegression()
lr_2.fit(X_train, y_train)

In [ ]:
lr_2.predict(X_test)

In [ ]:
lr_2.get_rmse(X_test, y_test)

In [ ]:
lr_2.get_r2_score(X_test, y_test)

In [ ]:
X_2, y_2 = pre_treatment(maison_2)

In [ ]:
X_2

In [ ]:
y_2

In [ ]:
X_2_train, X_2_test, y_2_train, y_2_test = train_test_split(X_2, y_2, test_size=0.1, random_state=1)
lr_3 = MyLinearRegression()
lr_3.fit(X_2_train, y_2_train)

In [ ]:
lr_3.predict(X_2_test)

In [ ]:
lr_3.get_rmse(X_2_test, y_2_test)

In [ ]:
lr_3.get_r2_score(X_2_test, y_2_test)